In [1]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv("final.csv")

In [7]:
df.head()

,qs,difficulty
0,"For 10 points, name this native of Rokovoko an...",college
1,It is more closely related to Dalmatian than t...,college
2,"For 10 points, name this state, home to Contin...",college
3,"For 10 points, name this author of ""The Fixati...",college
4,"For 10 points, identify this novel subtitled T...",college


In [8]:
df = df.dropna()

In [11]:
from nltk.corpus import stopwords

In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Kshitij
[nltk_data]     Alwadhi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [13]:
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

In [14]:
vocab_size = 5000 
embedding_dim = 64
max_length = 200
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>' # OOV = Out ofVocabulary

In [16]:
qs_list = []
labels = []

for idx,row in df.iterrows():
    labels.append(row['difficulty'])
    qs = row['qs']
    qs = qs.lower()
    qs = ' '.join(word for word in qs.split() if word not in STOPWORDS)
    qs_list.append(qs)
        

In [20]:
qs_list[4]

'10 points, identify novel subtitled modern prometheus, mary shelley work creating certain monster.'

In [21]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [22]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(qs_list)
word_index = tokenizer.word_index

In [27]:
qs_sequences = tokenizer.texts_to_sequences(qs_list)

In [28]:
qs_padded = pad_sequences(qs_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [29]:
qs_padded[1]

array([3405, 1512,    1,   90,  158,  237,  116,    1,    1,    1,    1,
          1,    1,    1,    1,    1,  287, 1208,    1,  948,  584,    4,
          3,    2,    1, 1999,  280,  964, 4374,    1,    1,  427, 1789,
        237, 1999,    1,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [30]:
print(set(labels))

{'high_school', 'college'}


In [50]:
# 0 for college, 1 for high school
tokenized_labels = []
for entry in labels:
    if entry == 'high_school':
        tokenized_labels.append(1)
    else:
        tokenized_labels.append(0)

In [51]:
from sklearn.model_selection import train_test_split

In [52]:
X_train, X_test, Y_train, Y_test = train_test_split(qs_padded,tokenized_labels, test_size = 0.10, random_state = 42)

In [45]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Dropout, Activation, Embedding, Bidirectional

In [46]:
model = Sequential()
model.add(Embedding(vocab_size,embedding_dim))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(embedding_dim)))
model.add(Dense(2,activation='softmax'))

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          320000    
_________________________________________________________________
dropout (Dropout)            (None, None, 64)          0         
_________________________________________________________________
bidirectional (Bidirectional (N

In [47]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)

In [54]:
epochs = 5
batch_size = 64
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

Train on 68636 samples, validate on 7627 samples
Epoch 1/5
  448/68636 [..............................] - ETA: 28:59 - loss: 0.6689 - acc: 0.5893

KeyboardInterrupt: 